In [1]:
from itertools import cycle

def generators():
    with open("Day17.txt") as file:
        jets = cycle(enumerate(c for c in file.read()))
    rocks = cycle(enumerate((
        lambda y: {(2, y + 4), (3, y + 4), (4, y + 4), (5, y + 4)},
        lambda y: {(3, y + 6), (2, y + 5), (3, y + 5), (4, y + 5), (3, y + 4)},
        lambda y: {(4, y + 6), (4, y + 5), (2, y + 4), (3, y + 4), (4, y + 4)},
        lambda y: {(2, y + 7), (2, y + 6), (2, y + 5), (2, y + 4)},
        lambda y: {(2, y + 5), (3, y + 5), (2, y + 4), (3, y + 4)},
    )))
    return jets, rocks

In [2]:
def move_left(rock, chamber):
    new_rock = {(x - 1, y) for x, y in rock}
    if any(x < 0 for x, y in new_rock) or new_rock & chamber:
        return rock
    return new_rock

def move_right(rock, chamber):
    new_rock = {(x + 1, y) for x, y in rock}
    if any(x > 6 for x, y in new_rock) or new_rock & chamber:
        return rock
    return new_rock

def move_down(rock, chamber):
    new_rock = {(x, y - 1) for x, y in rock}
    return rock if new_rock & chamber else new_rock

jet_move = {
    "<": move_left,
    ">": move_right,
}

In [3]:
def run(total_rocks):
    height = 0
    cycles = {}
    chamber = {(i, 0) for i in range(7)}
    jets, rocks = generators()
    for rock_number in range(total_rocks):
        rock_index, rock_builder = next(rocks)
        rock = rock_builder(height)
        while True:
            jet_index, direction = next(jets)
            cycle_key = (rock_index, jet_index)
            if cycle_key in cycles:
                previous_rock_number, previous_height = cycles[cycle_key]
                cycle_period = rock_number - previous_rock_number
                if rock_number % cycle_period == total_rocks % cycle_period:
                    cycle_height = height - previous_height
                    remaining_rocks = total_rocks - rock_number
                    remaining_cycles = (remaining_rocks // cycle_period) + 1
                    estimated_height = previous_height + (cycle_height * remaining_cycles)
                    print(f"Cycle of period {cycle_period} detected at iteration {previous_rock_number} / {rock_number}")
                    return estimated_height
            else:
                cycles[cycle_key] = (rock_number, height)
            rock = jet_move[direction](rock, chamber)
            new_rock = move_down(rock, chamber)
            if rock == new_rock:
                chamber.update(rock)
                height = max(y for x, y in chamber)
                break
            rock = new_rock
    return height

In [4]:
%%time
run(2022)

CPU times: user 685 ms, sys: 5.04 ms, total: 690 ms
Wall time: 692 ms


3191

In [5]:
%%time
run(1_000_000_000_000)

Cycle of period 1720 detected at iteration 1440 / 3160
CPU times: user 1.58 s, sys: 1.94 ms, total: 1.58 s
Wall time: 1.59 s


1572093023267